In [ ]:
import numpy as np
from time import time
import os
import sys
import scipy
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

sys.path.insert(0, '/home/users/brezovec/.local/lib/python3.6/site-packages/lib/python/')
import ants

from BigBadBrain.brain import bleaching_correction, z_score_brain, get_resolution, save_brain, load_numpy_brain, get_dims
from BigBadBrain.fictrac import load_fictrac, interpolate_fictrac
from BigBadBrain.utils import load_timestamps, get_fly_folders, send_email
from BigBadBrain.glm import fit_glm, save_glm_map, create_multivoxel_X_matrix
from BigBadBrain.motcorr import get_motcorr_brain, motion_correction

In [4]:
import BigBadBrain

In [13]:
bbb.load_numpy_brain?

In [14]:
import BigBadBrain as bbb2

In [5]:
dir(BigBadBrain)

['__doc__', '__loader__', '__name__', '__package__', '__path__', '__spec__']

In [2]:
dir(bbb.utils)

AttributeError: module 'BigBadBrain' has no attribute 'utils'